In [1]:
import torch
import ultralytics
model = torch.load('MicroVision-Chalmers-x-20250304.pt', weights_only=False)
import cv2
import os
import json
from ultralytics import YOLO  



In [ ]:

# Define your classes here (same order as during YOLO training)
classes = ["pedestrians", "bikes", "bicyclists", "e-scooters", "e-scooterists"]

# Load YOLO model
model = YOLO("MicroVision-Chalmers-x-20250304.pt")

video_name='GS010659'
# Open the video file
video_path = "Videos/" + video_name + ".mp4"
cap = cv2.VideoCapture(video_path)

# Get video frame dimensions and fps
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Create output directories if they don't exist
images_dir = "yolo/datasets_"+ video_name +"/one/images"
labels_dir = "yolo/datasets_"+ video_name +"/one/labels"
os.makedirs(images_dir, exist_ok=True)
os.makedirs(labels_dir, exist_ok=True)

# Create the classes.txt file
classes_file = "yolo/datasets_"+ video_name +"/one/classes.txt"
with open(classes_file, "w") as class_f:
    for cls in classes:
        class_f.write(f"{cls}\n")

print(f"Classes file created at: {classes_file}")

# Define the codec and create a VideoWriter object
output_path = "predictions/annotated_video_"+ video_name +".mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

frame_idx = 0  # Frame counter

# Process video frames
while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    # Run YOLO detection
    results = model.track(frame, persist=True, imgsz=1280,conf=0.1)

    # Visualize results on the frame
    annotated_frame = results[0].plot()

    # Save the current frame as an image for Label Studio
    image_file = os.path.join(images_dir, f"frame_{frame_idx:04d}.jpg")
    cv2.imwrite(image_file, frame)

    # Save detections in YOLO format
    label_file = os.path.join(labels_dir, f"frame_{frame_idx:04d}.txt")
    with open(label_file, "w") as f:
        for r in results[0].boxes:
            cls = int(r.cls)
            x_center, y_center, w, h = r.xywhn[0]  # normalized (x_center, y_center, width, height)

            # Write to YOLO label file
            f.write(f"{cls} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n")

          
      

    # Write the annotated frame to the output video
    out.write(annotated_frame)

    # Display the annotated frame (optional, can comment out)
    # cv2.imshow("Annotated Frame", annotated_frame)

    # Break loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

    frame_idx += 1  # Increment frame counter


# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()


SyntaxError: invalid syntax (1273976817.py, line 2)

In [52]:
import yaml

# Définition du dataset
dataset = {
    "train": "//Users/martin.dejaeghere/Microsint2/yolo/datasets/one",
    "val": "/Users/martin.dejaeghere/Microsint2/val",
    "nc": 5,  # Nombre de classes
    "names": ['pedestrians', 'bikes', 'bicyclists', 'e-scooterists', 'e-scooters']  # Classes
}

# Sauvegarde dans un fichier
with open("dataset.yaml", "w") as file:
    yaml.dump(dataset, file, default_flow_style=False)

print("Fichier dataset.yaml créé avec succès !")


# Validate with a custom dataset
metrics = model.val(data="dataset.yaml",conf=0.7)  # output optional
print(metrics.box.map)  # map50-95

Fichier dataset.yaml créé avec succès !
Ultralytics 8.3.87 🚀 Python-3.11.11 torch-2.6.0 CPU (Apple M3 Pro)


val: Scanning /Users/martin.dejaeghere/Microsint2/val/labels.cache... 56 images, 0 backgrounds, 0 corrupt: 100%|██████████| 56/56 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [02:37<00:00, 39.49s/it]


                   all         56       1673      0.769      0.258      0.515      0.456
                person         56       1320      0.979      0.145      0.563      0.519
               bicycle         34         36          0          0          0          0
               cyclist         53        163      0.866      0.356      0.619      0.586
             e-scooter         43         74          1      0.162      0.581      0.397
          e-scooterist         53         80          1      0.625      0.812      0.779
Speed: 1.4ms preprocess, 2789.2ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs/detect/val13
0.456353859799755


In [50]:
metrics.confusion_matrix.plot()  # normalized confusion matrix